In [ ]:
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from pydantic import BaseModel, Field, model_validator
from typing import List
import numpy as np

In [ ]:
class Schema(BaseModel):
    x: List = Field(description="First List", max_length=4, min_length=2)
    y: List = Field(description="Second List", max_length=4, min_length=2)

    @model_validator(mode="after")
    def ValidateDimensions(self):
        if not len(self.x) == len(self.y):
            raise ValueError(f"Invalid Dimensions {len(self.x)} != {len(self.y)}")
        return self


class MyEstimator(BaseEstimator):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        self.config = Schema(x=X["x"].tolist(), y=X["y"].tolist())
        result = np.array(
            [x + y for x, y in zip(self.config.x, self.config.y)]
        ).reshape(-1, 1)
        return result

In [93]:
trf = MyEstimator()

In [94]:
pipe = Pipeline(steps=[("Element Wise Addition", trf)])

In [95]:
pipe

Pipeline(steps=[('Element Wise Addition', MyEstimator())])

In [96]:
final_transformation = ColumnTransformer(
    transformers=[("Final Transformation", pipe, ["x", "y"])]
)

In [97]:
import pandas as pd

data = pd.DataFrame({"x": [1, 2, 3, 4], "y": [3, 4, 5, 6]})

In [104]:
try:
    res = final_transformation.fit_transform(data)
except Exception as e:
    print(e)
print(res)

[[ 4]
 [ 6]
 [ 8]
 [10]]
